# 신경망의 학습구현

##  손실함수 Loss Function

- 손실을 산출하기 위한 수식들을 구현한 함수
- 손실 Loss는 신경망이 예측한 결과와 실제 정답사이의 오차를 나타내는 단일 값(스칼라)
- 손실을 구하기 위해서는 먼저 은닉층을 거쳐 변환된 입력값들을 이용하여 softmax함수를 적용하여 새로운 값으로 반환한다
- 이 때의 값은 어떤 클래스들에 대한 소속 확률을 말한다

## Softmax 함수
https://wikidocs.net/35476
## $y_k = \frac{exp(s_k)}{\sum_{i = 1}^{n}exp(s_i)}$
- 출력이 n개일 때, k번째 출력 $y_k$, 분자는 K번째 클래스의 점수
- 분모는 모든 클래스들의 점수
- 따라서 결국 모든 클래스들의 소프트맥스 함수값을 더하면 1이 산출됨
- 때문에 소프트맥스 출력값을 특정 클래스에 속할 확률로 보는 것

## Cross Entropy 함수
https://wikidocs.net/35476
## $L = -\sum_{k}t_k\log y_k$
- 여기서 로그는 자연로그를 의미
- $t_k$는 k번째 클래스에 해당하는 정답 레이블 
- $t_k$가 **원-핫벡터**이므로 정답에 해당하는 클래스만의 오차를 구하게 됨
- 만약 $y_k$가 1이면 $log1 = 0$ 이므로 손실함수 값도 0
- 따라서 Cross Entropy함수의 값이 최소화되기 위해서는 $y_k = 1$이 되는, 즉 정확한 예측을 해야하므로
- 손실함수로써 적합함

## 미니배치 작동방식의 Cross Entropy 함수
## $L = -\frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log y_{nk}$
- 각 데이터, 케이스에 대해서 손실을 계산해야 하므로
- 데이터의 개수 n과 $\sum_{n}$이 추가 되고
- 평균 손실함수를 구하게 되므로 $\frac{1}{N}$이 추가됨

In [1]:
#Softmax 함수 구현
# https://github.com/WegraLee/deep-learning-from-scratch-2/blob/master/common/functions.py
import numpy as np


def softmax(x):
    if x.ndim >= 2: #입력이 2차원 이상일 때
        x = x - x.max(axis = 1, keepdims = True) #오버플로우를 막기 위해
        #예로 1000이라는 값이 들어가면 exp(1000)이 계산되지 못함
        #따라서 수들을 가장 큰 수를 바탕으로 각각 빼주고 계산을 수행
        x = np.exp(x)
        x /= x.sum(axis = 1, keepdims = True) #여기서 x는 np.exp(x)를 말함
        
    elif x.ndim == 1:
        x = x - np.max(x) #1차원이기 때문에 axis지정 필요가 없음
        x = np.exp(x) / np.sum(np.exp(x))
                
    return x

In [2]:
#Cross Entropy 함수 구현

def cross_entropy_error(y, t):
    #y는 소프트맥스 반환 값, t는 실제 정답데이터
    
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    if t.size == y.size:
        t = t.argmax(axis = 1) #정답 1인 인덱스 값을 반환
        #즉 만약 3개의 클래스가 있다면 나올 수 있는 인덱스는 0, 1, 2
        
    batch_size = y.shape[0]
    
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    # t_k * log y_k는 결국 y의 정답에 해당하는 클래스의 로그값만 산출하면 되기 떄문에
    # np.log안에서 y의 인덱싱을 사용하는 것
    # 행은 배치사이즈의 개수, 즉 케이스의 개수들을 사용 해야하기 때문에 
    # range와 같은 기능을 하는 np.arange를 사용
    # 열은 정답에 해당하는 클래스를 알아야 하므로 t.argmax를 사용하여 입력

## 미분과 기울기
- 신경망 학습의 목표는 손실을 최소화하는 매개변수params를 찾는것
- x에 대한 y의 미분값은 x의 변화에 따른 y의 변화량을 의미하며
- 이때의 변화량은 기울기를 의미
- 왜 기울기를 구하는가?  
> 우리가 기울기를 알게 되면 어떤 변수의 변화에 따라서 y가 변화하는 정도를 알 수 있게 되기 때문  
> 신경망에서의 기울기는 손실에 대한 기울기      
> 즉 각 가중치가 손실을 발생시키는것에 얼마나 영향을 미치는지를 말하는것     
> **기울기를 조정**하여 y의 변화의 정도를 달리할 때 결과를 비교하여서   
> 보다 정확한 y의 결과를 산출하도록 조정할 수 있을 것 -> 결국 기울기를 변동시키는일

## 신경망에서의 역전파
- 신경망 학습의 목적은 **손실을 최소화** 하는 것
- 손실을 최소화 하기 위해서는 매번 계산되는 오차를 줄이는 방향으로 학습이 되어야 함
- 신경망에서의 학습의 주요 파라미터는 가중치
- 즉 가중치들을 업데이트 해야 손실도 업데이트
- 그런데 가중치들마다 **손실에 영향을 주는 정도가 다를 것**
- 손실에 영향을 주는 정도는 기울기
- 신경망은 여러 함수들의 합성으로 이루어지는 거대 **합성함수**
- 따라서 기울기는 합성함수의 미분, 즉 연쇄법칙을 이용해서 기울기를 구할 수 있다
- 이렇게 연쇄법칙을 사용하면 최종 출력층에서 최초 입력층까지 가는 층들 사이의 모든 기울기들을 구할 수 있게 된다
- 이렇게 구해진 **기울기들에 대한 튜닝**이 이루어지는게 신경망에서의 학습이다

## 신경망 구현을 위한 다양한 노드들
- 덧셈노드: 어떤 계산된 값들을 더하는 역할 수행
- 곱셈노드: 어떤 계산된 값들을 곱하는 역할 수행
- 분기노드: 어떤 값들을 다음의 노드들로 복제하여 이동시키는 역할

In [3]:
# Repeat 노드 구현, 분기노드의 일종
# 즉 N개로 분기하는 노드가 Repeat 노드

D, N = 8, 7 #D는 차원 수, N은 케이스 개수

#순전파
#여기서 x는 가중치라고 가정함
x = np.random.randn(1, D) #반복하고자 하는 입력 x
y = np.repeat(x, N, axis = 0) #x를 케이스 개수 N만큼 반복하여 생성
    #행단위로 생성되기 때문에 axis = 0

#역전파
dy = np.random.randn(N, D)
dx = np.sum(dy, axis = 0, keepdims = True)

In [4]:
x

array([[ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ]])

In [5]:
y

array([[ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ],
       [ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ],
       [ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ],
       [ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ],
       [ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ],
       [ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ],
       [ 0.35832679,  1.42660931,  0.42843892, -0.62493969,  0.97154252,
         0.06677258,  0.22583876, -0.0750021 ]])

In [6]:
dy

array([[-1.58705079,  0.86673324, -0.05114883,  0.83979214, -0.0545844 ,
         2.25655951, -2.14715897,  0.98560426],
       [-0.4927982 ,  1.52041027,  0.09259118, -1.5413771 , -0.28601281,
         0.6477942 ,  0.11908493,  0.93688839],
       [-0.28496575, -0.8571765 ,  0.45112044, -0.37726422, -0.664151  ,
         0.5211662 ,  1.18862862,  1.00632612],
       [-2.08689169,  0.16120828,  2.15293996,  0.79785055,  1.74885718,
        -2.26928078, -1.07280458,  1.12038807],
       [-1.71690516, -0.66746499,  0.65167683,  0.97956274,  1.42153392,
        -0.91776403, -0.11173555,  0.08144114],
       [ 0.84242643, -0.63090211,  0.16342902, -0.32291178,  0.01346453,
        -1.08847955,  0.43452648, -0.81062004],
       [-0.01574223, -0.80162899, -0.40268858,  0.62488733,  1.91477952,
        -0.71480347, -0.65221657,  1.68182359]])

In [7]:
dx

array([[-5.3419274 , -0.40882081,  3.05792003,  1.00053965,  4.09388694,
        -1.56480791, -2.24167564,  5.00185153]])

In [8]:
#Sum 노드
#Repeat노드의 반대

#순전파
D, N = 8, 7

x = np.random.randn(N, D)
y = np.sum(x, axis = 0, keepdims = True)

#역전파
dy = np.random.randn(1, D)
dx = np.repeat(dy, N, axis = 0)

In [9]:
x

array([[ 0.65290865, -0.11929516,  0.85045447, -0.746642  , -0.22061344,
        -0.06053845, -0.50850599, -1.26458318],
       [ 0.92851761, -1.1535945 ,  1.47765004, -1.03468855, -0.54629744,
        -1.10727647,  0.48388698, -0.07466376],
       [ 0.30421126, -0.05080099,  1.44816276,  0.79658339, -0.84989139,
         0.62706366,  0.32516328,  0.4687517 ],
       [ 0.46598034,  1.66151729, -2.7459132 , -0.83415049, -0.37079479,
         0.70181227, -2.1096123 , -0.09015812],
       [-0.63646922, -0.32474407, -0.91905948, -0.45409248,  0.66499036,
        -1.41825484,  0.58028088, -2.06765433],
       [ 0.13619767, -0.30516651,  0.03760699,  2.69256849, -0.15980494,
        -0.4350568 ,  1.96331884, -0.39232803],
       [ 0.51254539, -0.23737964,  0.17545838,  0.6036519 ,  0.0518461 ,
        -0.02517837,  0.25615402, -1.21947134]])

In [10]:
y

array([[ 2.36389169, -0.52946358,  0.32435996,  1.02323025, -1.43056555,
        -1.717429  ,  0.99068571, -4.64010708]])

In [11]:
dy

array([[ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715]])

In [12]:
dx

array([[ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715],
       [ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715],
       [ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715],
       [ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715],
       [ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715],
       [ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715],
       [ 0.86903658, -0.32281983,  1.23746052,  0.43589973,  0.60242731,
         0.04704064,  1.78499222, -0.46355715]])

In [13]:
# Matmul 노드, 곱셈 노드 구현
#입력 X와 가중치 W를 곱하는 곱셈노드 
import numpy as np

class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None #초기 입력은 없는 상태
        
    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x #인스턴스 변수에 X를 할당
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T) #x의 역전파
        dW = np.dot(self.x.T, dout) #W의 역전파
        self.grads[0][...] = dW #여기서 [...]는 덮어쓰기 기능
        #덮어쓰기는 처음 변수가 할당된 메모리 주소를 그대로 이용하는 것
        #즉 처음 변수가 주어진 메모리에서 해당 변수의 데이터만 바꾸는 것
        #따라서 변수 할당을 또 다른 메모리에 할 필요가 없으므로 가중치 업데이트에서 효율적임
        return dx

In [14]:
#시그모이드 계층 순전파, 역전파 구현

class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [15]:
#순전파 계층 구현
#지금까지 구현한 Matmul, Sigmoid, Sum, Repeat을 사용

class Affine:
    def __init__(self, W, b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)] #초기 가중치값 초기화
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b #여기서 b는 브로드캐스팅을 시행
        #브로드캐스팅을 위해 b들이 repeat되므로 repeat노드 역할이 이뤄지는 것
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(x.T, dout)
        db = np.sum(dout, axis = 0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [16]:
#가중치 갱신
#확률적 경사하강법 SGD 이용
#확률적 경사하강법: 무작위로 선택된 데이터, 즉 미니배치에 대한 기울기를 이용하여 가중치 갱신
#이 때 가중치 갱신은 

class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

## 확률적 경사하강법
https://www.youtube.com/watch?v=IHZwWFHWa-w&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=2&ab_channel=3Blue1Brown

- 역전파로 계산된 가중치들은 다음을 나타낸다
- 가중치의 부호는 해당 가중치들이 업데이트 될 때 커저야 하는지, 작아져야 하는지를 나타낸다
- 만약 기울기가 양의 부호이면 해당 가중치가 작아져야 한다는 것이고, 
- 기울기가 음의 부호이면 해당 가중치가 커져야 한다는 것이다
- 가중치의 절대값 크기는 해당 가중치들이 얼마나 중요하게 업데이트 되어야 하는지를 나타낸다
- 만약 기울기의 절대값이 크다면 해당 가중치는 보다 많이 업데이트 되어야 하고
- 기울기의 절대값이 작다면 해당 가중치는 보다 적게 업데이트 되어야 한다
- 그리고 이 때 가중치 업데이트의 속도는 학습률, Learning Rate가 결정한다

## DNN의 함정
- Cost Function은 단순히 현재 주어진 데이터셋의 라벨값을 바탕으로 정답인지 아닌지를 가린다
- 만약 기존의 라벨링이 없는 새로운 데이터가 DNN에 들어올 때 (혹은 라벨링이 우리가 가진 데이터셋 라벨의 범주를 벗어나는 경우)
- DNN은 학습을 통해서 어떠한 결과를 도출해 낼 것이다
- 그러나 해당 결과는 연구자의 기대와 다른 전혀 다른 결과이다 (우리는 애초에 input단계에서 데이터가 다른 형태임을 인지할 것이라고 기대하지만 그렇지 않다)
- 그 이유는 Cost Function이 단순히 초기 랜덤하게 주어진 가중치들을 가지고 최소화만을 추구하는 기법을 다루기 때문이다
- 기존의 우리가 가지고 있는 데이터셋은 정답과 오답을 명확하게 구분할 수 있어 Cost의 측정에 부합하고 이를 바탕으로 Cost Fucntion을 이용하여 최소화를 추구할 수 있다
- 그러나 전혀 다른 형태의 데이터가 DNN에 투입되게 된다면 오답을 가릴 수 없으므로 Cost Funcnction이 제대로 작동하지 못한다
- 따라서 기존의 데이터셋과 전혀 다른 데이터가 DNN에 투입될 경우 DNN은 제대로 성능을 낼 수 없으며
- 설령 어떤 성능을 내더라도 그것이 제대로 학습을 했다고 볼 수 없다

위에서 밝혀진 한계로 이후 다양한 신경망 기법들이 등장하게 되었다고 볼 수 있다
- 보다 정확한 신경망이 되기 위해서는 Cost Function이 정교해져야하거나
- 초기 input 단계에서 데이터를 거를 수 있는 방법이 개발되어야 할 것이다